In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers.experimental import preprocessing
import pandas as pd
import os
import datetime
from unidecode import unidecode
import time

In [ ]:
dataset_url = "https://github.com/darksun-misc/piratebay-db-dump/raw/master/piratebay_db_dump_2004_03_25T22_03_00_to_2015_10_27T04_10_22.csv"
dataset_raw = pd.read_csv(dataset_url, sep=";", error_bad_lines=False)
text = unidecode('\n'.join(list(dataset_raw['NAME'].astype(str))))
vocab = sorted(set(text))

In [ ]:
ids_from_chars = preprocessing.StringLookup(vocabulary = list(vocab))
chars_from_ids = preprocessing.StringLookup(vocabulary = ids_from_chars.get_vocabulary(), invert = True)
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8', 'ignore'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length+1)

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

split_input_target = lambda sequence: (sequence[:-1], sequence[1:])

dataset = sequences.map(split_input_target)

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    sequences
    .map(split_input_target)
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [ ]:
vocab_size = len(ids_from_chars.get_vocabulary())

embedding_dim = 256

rnn_units = 2048

class TPBGen(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)
        
    def call(self, inputs, states = None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training = training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)
        
        if return_state:
            return x, states
        else:
            return x

In [ ]:
model = TPBGen(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS = 20

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature=temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices = skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())]) 
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
    predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
start = time.time()
states = None
next_char = tf.constant(['['])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()

results_string = result[0].numpy().decode('utf-8')

with open('output.txt', 'w') as f:
    f.write(results_string)

print(f"\nRun time: {end - start}")